Imports and endpoints

In [44]:
import requests
import pandas as pd
pd.set_option('display.max_columns', 100)
from time import sleep

urls = {
    'api' : 'https://api.mtga.untapped.gg/api/v1/',
    'json' : 'https://mtgajson.untapped.gg/v1/latest/',
}

endpoints = {
    'active': ('api', 'meta-periods/active'),
    'tags': ('api', 'tags'),
    'archetypes': ('api', 'analytics/query/archetypes_by_event_scope_and_rank?MetaPeriodId=334&RankingClassScopeFilter=BRONZE_TO_PLATINUM'),
    'analytics': ('api', 'analytics/query/card_stats_by_archetype_event_and_scope_free/ALL?MetaPeriodId=334'),
    'cards': ('json', 'cards.json'),
    'text': ('json', 'loc_en.json'),
}

headers = {
    'authority': 'api.mtga.untapped.gg',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,pt;q=0.8',
    'if-none-match': '"047066ff947f01e9e609ca4cf0d6c0a6"',
    'origin': 'https://mtga.untapped.gg',
    'referer': 'https://mtga.untapped.gg/',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}

def request_json(keyword, send_headers=True):
    'Returns JSON from corresponding keyword'

    url_kw, endpoint = endpoints[keyword]
    url = urls[url_kw]
    
    sleep(2)
    if send_headers:
        return requests.get(url+endpoint, headers).json()
    else:
        return requests.get(url+endpoint).json()

Active

In [2]:
def get_legal_sets():
    'Returns lists of standard legal sets'
    latest = dict()

    # Extracting information from the lastest standard BO1 format
    for format in request_json('active'):
        if format['event_name'] == 'Ladder':
            latest = format

    return latest['legal_sets']

# get_legal_sets()

Cards

In [3]:
def get_cards():
    'Returns transformed cards dataframe'
    df = pd.DataFrame(request_json('cards'))

    # Ony bother with standard legal cards
    df = df[df.set.isin(get_legal_sets())]

    # Remove duplicates by considering only the latest reprint
    gb = df.groupby('titleId').agg({'grpid':'max'})
    df = df[df.grpid.isin(gb.grpid)]
    # print(df.titleId.is_unique)

    return df.set_index('grpid')

# cards = get_cards()

Text

In [4]:
def get_text():
    'Returns card text dataframe'

    df = pd.DataFrame(request_json('text')).set_index('id')

    # Collapse columns raw and text, prioritizing raw
    df.loc[~df.raw.isna(), 'text'] = df.raw
    df.drop('raw', axis='columns', inplace=True)

    return df

# text = get_text()

Card information

In [38]:
def get_card_information():
    text = get_text()
    cards = get_cards()

    # Card columns mappable to text
    text_columns = ['titleId', 'flavorId', 'cardTypeTextId', 'subtypeTextId']

    # Additional transformed columns
    transformed = (
        cards[text_columns]
        .applymap(lambda x: text.loc[x].values[0], na_action='ignore')
        .rename(columns={column: column[:-2] for column in text_columns}) # Remove Id
    )
    cards = cards.join(transformed)

    # Remove undesirable columns
    cards = cards.dropna(how='all', axis='columns')
    ignore = [ # These don't seem useful
        'collectorMax', 
        'frameColors',
        'rawFrameDetails',
        'frameDetails',
    ]
    problematic = [ # Still work to be done here
        'abilities', 
        'abilityIdToLinkedTokenGrpId', 
        'hiddenAbilities',
        'types',
        'subtypes',
        'supertypes',
        'colors',
        'colorIdentity',
        'linkedFaces',
    ]
    cards = cards.drop(ignore + problematic, axis = 'columns')

    return cards

card_information = get_card_information()

In [39]:
card_information.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2158 entries, 78323 to 87130
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titleId             2158 non-null   int64  
 1   artId               2158 non-null   int64  
 2   flavorId            2158 non-null   int64  
 3   artistCredit        2156 non-null   object 
 4   power               1274 non-null   object 
 5   toughness           1305 non-null   object 
 6   set                 2158 non-null   object 
 7   collectorNumber     2158 non-null   object 
 8   castingcost         1864 non-null   object 
 9   rarity              2093 non-null   float64
 10  cardTypeTextId      2158 non-null   int64  
 11  subtypeTextId       1501 non-null   float64
 12  altDeckLimit        5 non-null      float64
 13  linkedFaceType      283 non-null    float64
 14  isSecondaryCard     651 non-null    object 
 15  isToken             65 non-null     object 
 16  IsDigi

In [41]:
# for column in card_information.columns:
#     print(column)
#     print('\t', card_information[column].nunique())

card_information.nunique()

titleId               2158
artId                 2069
flavorId              1015
artistCredit           487
power                   12
toughness               15
set                      7
collectorNumber        427
castingcost            202
rarity                   5
cardTypeTextId          24
subtypeTextId          439
altDeckLimit             1
linkedFaceType           5
isSecondaryCard          1
isToken                  1
IsDigitalOnly            1
watermark               26
RebalancedCardLink     178
altTitleId               7
IsRebalanced             1
title                 2158
flavor                1015
cardTypeText            24
subtypeText            439
dtype: int64

Analytics

In [46]:
analytics_json = request_json('analytics', False)
analytics = analytics_json['data']
metadata = analytics_json['metadata']

In [47]:
normalized = pd.json_normalize(
    analytics,
).T
normalized

,0
763.ALL.b,"[[113, 50, 113, [19, 42, 25, 27]]]"
763.ALL.g,"[[244, 134, 244, [16, 111, 69, 48]]]"
763.ALL.p,"[[493, 263, 493, [44, 279, 96, 74]]]"
763.ALL.s,"[[182, 87, 182, [41, 94, 13, 34]]]"
953.510.b,"[[104, 52, 104, [2, 19, 24, 59]]]"
...,...
683790.510.p,"[[134, 47, 134, [54, 36, 22, 22]]]"
683790.2600.p,"[[105, 47, 105, [47, 28, 13, 17]]]"
683793.ALL.p,"[[128, 49, 128, [36, 77, 3, 12]]]"
683798.ALL.g,"[[107, 54, 107, [74, 33]]]"
